In [1]:
import sys
import os
from pathlib import Path

%reload_ext autoreload
%autoreload 2

base_path=Path("d:/")
base_path = base_path / "work" / "projects"
for pkg in ['ezai_util','ai_coop_py']:
    pkg_path = base_path / pkg
    pkg_path = str(pkg_path.resolve())
    print(pkg_path)
    if not pkg_path in sys.path:
        sys.path.append(pkg_path)
import ezai_util
import airsim
from ezai_util import DictObj

D:\work\projects\ezai_util
D:\work\projects\ai_coop_py


In [2]:
uenv_path = Path("d:/") / 'work' / 'unity-envs'
uenv_file = uenv_path / "berlin_walk_ml_beta" / 'Berlin_ML'
uenv_file = str(uenv_file.resolve())
#uenv_file = None  # if you want to use editor then uncomment it

In [3]:
rl_conf = DictObj({
  "env_name":'airsim',
  "seed": 123,
  "discount": 0.99,
  "tau": 5e-3,
  "expl_noise": 0.1,
  "batch_size": 256,
  "batches_before_train": 2,
  "memory_capacity":10000,
  "episode_max_steps":100,
  "num_episodes":2
})

env_conf = DictObj({
    'filename' : uenv_file,
    'log_folder' : '/tmp/unity',  # this folder should exist otherwise mono wont let the env load
    'seed' : rl_conf.seed,
    'observation_mode' : 0,
    'max_steps' : rl_conf.episode_max_steps,
    'reward_for_goal' : 50,
    'reward_for_ep': .005,
    'reward_for_other' : -.1,
    'reward_for_falling_off_map' :  -50,
    'reward_for_step': -.0001,
    'segmentation_mode': 1
})

In [4]:
env = airsim.AirSimEnv(env_conf) #filename=uenv_file, observation_mode = 0, max_steps = rl_conf.episode_max_steps)
env.info()

2020-10-16 19:51:00 INFO [environment.py:107] Connected to Unity environment with package version 1.1.0-preview and communication version 1.0.0
2020-10-16 19:51:01 INFO [environment.py:265] Connected new brain:
VectorNavigator?team=0


Env Info
-----------
Action Space: Box(2,)
Action sample: [ 0.39293838 -0.42772132]
Action Space Shape: (2,)
Action Space Low: [-1. -1.]
Action Space High: [1. 1.]
Observation Mode: 0
Gym Observation Space: Tuple(Box(13,))
Gym Observation Space Shape: None
Self Observation Space: Tuple(Box(13,))
Self Observation Space Shape: None
Reward Range: (-inf, inf)
Metadata: {'render.modes': ['rgb_array']}
Initial State: [array([ 2.4645159e+03,  3.6936260e+01,  1.4888254e+03, -4.2447590e-04,
       -2.4768433e-01, -4.2742761e-03,  3.5542715e-04,  9.9996179e-01,
        4.9600005e-04,  8.7281242e-03,  2.4469949e+03,  3.9943001e+01,
        2.0894702e+03], dtype=float32)]
First Step State: ([array([ 2.4645159e+03,  3.6932774e+01,  1.4888258e+03,  2.2410507e-04,
       -2.0867470e-01,  2.5496276e-02,  9.1675203e-04,  9.9996078e-01,
        6.6970289e-04,  8.7882504e-03,  2.4469949e+03,  3.9943001e+01,
        2.0894702e+03], dtype=float32)], -1e-04, False, {'step': <mlagents_envs.base_env.DecisionS

c:\Miniconda3\envs\ezai\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
trainer = airsim.Trainer(rl_conf,env,enable_logging=False)

2020-10-16 19:51:03 WARNING [__init__.py:270] Could not seed environment VectorNavigator?team=0


In [6]:
trainer.train()

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.08it/s]


In [7]:
trainer.agent.save_actor('model.onnx')

In [8]:
env.close()

2020-10-16 19:51:05 INFO [environment.py:417] Environment shut down with return code 0 (CTRL_C_EVENT).
